Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Taxi"

In [137]:
import numpy as np
import random
from taxi_env_extended import TaxiEnvExtended

In [138]:
env = TaxiEnvExtended()

Obtener la cantidad de estados y acciones

In [139]:
actions = env.action_space.n
states = env.observation_space.n

Inicialización de la tabla Q

In [140]:
Q = np.zeros((states, actions))
Q

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [141]:
len(Q[0])

6

Obtención de la acción a partir de la tabla Q

In [142]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

Epsilon-Greedy Policy

In [143]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
    
    else: # exploit
        action = np.argmax(Q[state])
        
    return action

Ejemplo de episodio 

In [144]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
step_count = 0
while not done:
    state = obs
    action = epsilon_greedy_policy(state, Q, 0.5)
    obs, reward, done, _, _ = env.step(action)
    total_reward += reward
    step_count += 1
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)
print('total_steps', step_count)

367
exploit
-> 367 0 -1 467 False
explore
-> 467 2 -1 487 False
explore
-> 487 0 -1 487 False
explore
-> 487 3 -1 467 False
exploit
-> 467 0 -1 467 False
explore
-> 467 2 -1 487 False
exploit
-> 487 0 -1 487 False
exploit
-> 487 0 -1 487 False
explore
-> 487 0 -1 487 False
explore
-> 487 2 -1 487 False
explore
-> 487 5 -10 487 False
explore
-> 487 0 -1 487 False
explore
-> 487 1 -1 387 False
explore
-> 387 4 -10 387 False
explore
-> 387 5 -10 387 False
explore
-> 387 2 -1 387 False
explore
-> 387 2 -1 387 False
exploit
-> 387 0 -1 487 False
explore
-> 487 1 -1 387 False
explore
-> 387 0 -1 487 False
exploit
-> 487 0 -1 487 False
exploit
-> 487 0 -1 487 False
explore
-> 487 1 -1 387 False
explore
-> 387 2 -1 387 False
explore
-> 387 3 -1 367 False
exploit
-> 367 0 -1 467 False
exploit
-> 467 0 -1 467 False
exploit
-> 467 0 -1 467 False
explore
-> 467 5 -10 467 False
explore
-> 467 1 -1 367 False
explore
-> 367 5 -10 367 False
explore
-> 367 3 -1 367 False
exploit
-> 367 0 -1 467 False
e

Generar muchos episodios

In [145]:
alpha= 1 /env.max_steps
alpha

0.005

In [146]:
gamma = 0.9

In [147]:
episodes = 1000

In [148]:
i = 0
while i< episodes:
    obs, _ = env.reset()
    done=False
    total_reward = 0
    step_count = 0
    while not done:
        state = obs
        action = epsilon_greedy_policy(state, Q, 0.5)
        obs, reward, done, _, _ = env.step(action)
        
        total_reward += reward
        step_count += 1            
        value = Q[state][action] + alpha*(reward + gamma*np.argmax(Q[obs]) - Q[state][action])
        Q[state][action] = value
        env.render()

    i=i+1


exploit
exploit
exploit
exploit
exploit
explore
exploit
exploit
explore
exploit
exploit
explore
explore
explore
explore
exploit
exploit
exploit
explore
exploit
exploit
exploit
exploit
explore
exploit
explore
exploit
exploit
explore
exploit
explore
exploit
exploit
exploit
explore
explore
explore
exploit
exploit
exploit
explore
exploit
explore
exploit
explore
exploit
exploit
explore
explore
explore
explore
explore
explore
explore
exploit
exploit
exploit
exploit
exploit
explore
exploit
exploit
exploit
exploit
exploit
exploit
exploit
explore
explore
exploit
exploit
explore
exploit
explore
explore
explore
exploit
exploit
exploit
explore
explore
explore
explore
exploit
explore
exploit
explore
explore
exploit
exploit
exploit
exploit
explore
explore
explore
explore
explore
exploit
explore
exploit
exploit
exploit
explore
explore
exploit
explore
exploit
explore
explore
exploit
exploit
exploit
exploit
exploit
explore
exploit
exploit
explore
exploit
explore
explore
explore
exploit
exploit
exploit


In [149]:
Q

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.86912089e-01, -5.00000000e-03,  3.50250000e-03,
        -9.97500000e-03,  3.52500000e-03, -3.93069564e-01],
       [-9.97500000e-03,  6.92517506e-03,  7.48498750e-03,
         0.00000000e+00,  4.73125631e-02,  0.00000000e+00],
       ...,
       [ 2.04473381e-02, -1.98504994e-02,  3.72008759e-02,
         3.08855794e-01, -1.93916120e-01, -2.64799307e-01],
       [ 4.69066070e-01, -4.91704926e-01,  7.99897010e-01,
         8.67621824e-01, -4.45374325e+00, -4.60940182e+00],
       [-5.00000000e-03,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])